# Introduction
<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to perform Public Key rotation securely on TFLXTLS device.

The CryptoAuthentication device supports updating the public key after it has been validated. The public keys that are intended to rotate securely should be stored in slots marked as PubInvalid. 

The slot validation and invalidation is the process in the which the authenticity of the slot will be verified using its digest and signature provided by validation authority. Once its the authenticated, slot can be marked as validated or invalidated.

Once the key is validated a new key cannot be written before it is invalidated.

Secure Public key rotation is a multi step process,
1. Establish the communication between host(PC) and TrustFLEX
2. Generating a Validation Authority Key pair
3. Rotate new public key
4. Validate and verify rotated public key is usability

Before running this Notebook,
1. Make sure CryptoAuth Trust Platform is having factory reset program

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>



3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange> azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd()))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))
from trustplatform import program_flash
programmer = program_flash()

### Setup Modules and Hardware
This step loads the required modules and helper functions to perform the resource generation sequence. It also 
1. Initializes the interface with TFLXTLS hardware and establishes commmunication with TFLXTLS.
2. Performs device initialization to verify the actual device attached

In [ ]:
import os
import argparse,time
import binascii
import warnings

from cryptoauthlib import *
from public_key_rotation_helper import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptoauthlib.device import *
from cryptoauthlib.library import ctypes_to_bytes
from trustplatform import *

layout=widgets.Layout(width='auto')
warnings.filterwarnings('ignore')
print("Importing modules - Successful")

#TFLXTLS device I2C address 0x6C; #TNGTLS device I2C address 0x6A; 
common_helper.connect_to_secure_element('ATECC608A', ATCAKitType.ATCA_KIT_I2C_IFACE, 0x6C)
print("Device initialization - Successful")

### Generating a Validation Authority key pair
The most important step is to have the Validation Authority ECC P256 key pair. The key pair can be generated using many of the options available like openssl (or) python modules etc..,. The private key should be securely stored by OEM and the corresponding public key should be written to TFLXTLS devices.

<font color=red>This step is already perfomed part of resource generation using TFLXTLS_resource_generation\TFLXTLS Resource Generator.ipynb. </font> This key pair will be used for this public key rotation. 

<font color = green>The keys are generated here using python module and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage as depitected below.</font>

<center><img src="img/1.png" alt="**Get your Secure Elements here!**" /></center>

### Rotate new public key
The PubInvalid slots update is possible only after invalidating the slot i.e. only invalidated slots are allowed to write.

#### a. Generating a new public key
Generate a new key pair for system to rotate the existing public key with this new one. The new pubic key will be authorized by the validation authority. It calculates the new public keys digest and signs the same.

<center><img src="img/2-3-a.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def generate_key_pair(b):
    print ('\nGenerate new rotating public key pair...', end='')
    generate_new_rotate_public_key_pair()
    rotating_key_pair_generate.button_style = 'success'
    print('OK')

rotating_key_pair_generate = widgets.Button(description = "Step1. Gen Rotating key pair", 
    tooltip = 'Generate new rotating private/public key pair', layout=layout)
rotating_key_pair_generate.on_click(generate_key_pair)

#### b. Authorize public key
To validate or invalidate, its required to have Public keys digest and signature provided by validation authority. This ensures the key is coming from the right source. Validation authority calculates the digest and signs it. Authority public key on the product verifies the same as part of validate/invalidate process.

<center><img src="img/2-3-b.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
nonce = os.urandom(32)
rotating_public_key_digest = bytearray(32)
signature = bytearray(64)

def Authority_key_validate_rotate_public_key(b):
    global signature
    global nonce
    
    print ('\nSign the rotating key digest with the Authority Private Key...', end='')
    authority_private_key = get_validating_authority_key()
    rotating_private_key = get_rotating_key()
    rotating_public_key = rotating_private_key.public_key().public_bytes(encoding=Encoding.X962, format=PublicFormat.UncompressedPoint)[1:]
    rotating_public_key_digest = public_key_digest(nonce, rotating_public_key)
    signature = sign_host(rotating_public_key_digest, authority_private_key)
    print('OK')
    Authorize.button_style = 'success'
    
    resource_generate()
        
Authorize = widgets.Button(description ='Step2. Authorize new key', 
    tooltip='Authorize the new public key generated', layout=layout)
Authorize.on_click(Authority_key_validate_rotate_public_key)

#### c. Update public key
If a Public key is already stored in the slot and is validated, unless its invalidated it is not possible to overwrite the slot. Once it is invalidated, the new pubic key can be written to slot.

#### d. Validate the rotating public key
The new public key is updated after invalidating the existing key. Once the new key is written to the slot, its important to validate it. The successful validation restricts further writes to the slot until its invalidated. Also, after validating the slot can be used for cryptographic operations. 

The validation authority public key in the device verifies the signature to validate the slot.

<center><img src="img/2-3-d.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def rotate_public_key_write_and_validate(b):
    valid_buf = bytearray(4)
    global signature
    global nonce
    
    print('\nInvalidate public key slot...', end='')
    assert atcab_read_zone(ATCA_ZONE_DATA, rotating_key_slot, 0, 0, valid_buf, 4) == Status.ATCA_SUCCESS
    if(valid_buf[0] >> 4 == 0x05):
        assert public_key_invalidate() == 1
    print ('OK')

    print('Write rotating public key to device...', end='')
    rotating_private_key = get_rotating_key()
    rotating_public_key = rotating_private_key.public_key().public_bytes(encoding=Encoding.X962, format=PublicFormat.UncompressedPoint)[1:]
    assert atcab_write_pubkey(rotating_key_slot,rotating_public_key) == Status.ATCA_SUCCESS, 'Writing Public Key failed'
    print('OK')
    
    print ('Validate rotating public key with Authority signature...', end='')
    assert public_key_validate_from_authority(nonce, signature)
    print('OK')  
    write_and_validate_key.button_style = 'success'

write_and_validate_key = widgets.Button(description='Step3. Validate Slot for Rotation', 
    tooltip='Write and validate rotating public key', layout=layout)
write_and_validate_key.on_click(rotate_public_key_write_and_validate)

### Verify the rotating public key
This step verifies the rotating public written to device is validated and can be used for cryptographic operations.
The rotating private key signs a message on the host and is verified with the rotating public in slot

<center><img src="img/2-4.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def verify_rotate_public_key(b):
    is_verified_status = AtcaReference(False)

    message = os.urandom(32)
    print ('\nSign with the rotating private Key...', end='')
    rotating_private_key = get_rotating_key()
    signature = sign_host(message,rotating_private_key)
    print('OK')

    print('Verify the rotating public key...', end='')
    status = atcab_verify_stored(message, signature, rotating_key_slot, is_verified_status)
    if(is_verified_status == True):
        print('Success')
        verify.button_style = 'success'
    else:
        print('Failed')
        verify.button_style = 'danger'

    public_key_invalidate()
        
verify = widgets.Button(description = "Step4. Verify using Rotated key", 
    tooltip='Perform Verify with rotated Public Key', layout=layout)
verify.on_click(verify_rotate_public_key)
display(widgets.VBox((rotating_key_pair_generate, Authorize, write_and_validate_key, verify)))